<a href="https://colab.research.google.com/github/YunyyYY/xianyu/blob/master/novel_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [56]:
%cd gdrive/My\ Drive/messy

/content/gdrive/My Drive/messy


In [0]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [0]:
index = requests.get('https://www.luoxia.com/tianguancifu/')
soup = BeautifulSoup(index.content, 'html.parser')

In [0]:
mylist = soup.findAll("div", {"class": "book-list clearfix"})
mylist = mylist[0]
lis = mylist.find_all('a')

In [0]:
title = []
for a in lis:
    title.append(a.get('title'))

In [0]:
urls = []
for a in lis:
    # get content links
    urls.append(a.get('href'))

In [0]:
raw_text = []
for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    raw_text.append(soup.find(id='nr1'))

### Parse each chapter

In [0]:
def get_text(raw):
    # remove the ad containers
    x  = raw.find_all('p')
    content = ""
    for sen in x:
        tmp = "".join(sen.findAll(text=True))
        # remove the insrted website logo
        if bool(re.search('.*落.*霞.*小.*说.*', tmp)) == False:
            content += tmp
    return content

In [0]:
chapters = []
for x in raw_text:
    chapters.append(get_text(x))

In [46]:
len(chapters)

230

### Generate document

In [49]:
!pip install python-docx

     |████████████████████████████████| 5.5MB 4.6MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.10-cp36-none-any.whl size=184491 sha256=df4b231b74a46e895b761842dd6403e1ce4cb0a96dd8e8f311a3b2b877e98299
  Stored in directory: /root/.cache/pip/wheels/18/0b/a0/1dd62ff812c857c9e487f27d80d53d2b40531bec1acecfa47b
Successfully built python-docx


In [0]:
from docx import Document
from docx.shared import Inches

In [0]:
document = Document()

h = document.add_heading('天官赐福', 0)
h.alignment = 1
document.add_paragraph('\n')
for i, t in enumerate(title):
    h = document.add_heading(t, 1)
    h.alignment = 1
    document.add_paragraph('\n')
    document.add_paragraph(chapters[i])
    document.add_paragraph('\n')


In [0]:
document.save('tgcf.docx')